### 库导入

In [25]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

In [26]:
from lib.imgproc import to_jpeg_bytes, process_frame

### 全局变量

In [27]:
playing = False
frame_lock = threading.Lock()

### 自定义函数

In [28]:
# 播放线程：受全局变量 playing 控制
# 加上进程锁，一次只能一个访问这段代码
def play_video():
    global playing
    while playing:
        with frame_lock:
            ret, frame = cap.read()
            # 如果播放完毕，从头开始播放
            #-----------------------有限帧率的循环播放处理----------------------
            if not ret:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            # 播放的时候，对每一帧进行处理
            temp = [
                [slider_red1_h.value[0], slider_red1_s.value[0],slider_red1_v.value[0]],
                [slider_red1_h.value[1], slider_red1_s.value[1],slider_red1_v.value[1]],
                [slider_red2_h.value[0], slider_red2_s.value[0],slider_red2_v.value[0]],
                [slider_red2_h.value[1], slider_red2_s.value[1],slider_red2_v.value[1]],
            ]
            processed, mask = process_frame(frame, temp, lowest=slider_lowest.value)
            image_processed.value = to_jpeg_bytes(processed)
            image_mask.value = to_jpeg_bytes(mask, is_mask=True)
        # --------------------------控制视频的播放速度-----------------------
        time.sleep(0.001)

# 播放/暂停按钮逻辑
# toggle:切换
def toggle_play(change):
    global playing
    playing = change['new']
    # 如果正在播放,icon为pause,否则为play
    if playing:
        toggle_button.icon = 'pause'
    else:
        toggle_button.icon = 'play'
    # -----------------------------等待改进点，如何加强线程管理---------------------
    if playing:
        threading.Thread(target=play_video, daemon=True).start()

# 关闭按钮逻辑
def close_video(b):
    # 全部关闭
    global playing
    playing = False
    toggle_button.value = False
    toggle_button.disabled = True
    close_button.disabled = True
    slider_red1_v.disabled = True
    slider_red2_v.disabled = True
    slider_red1_h.disabled = True
    slider_red2_h.disabled = True
    slider_red1_s.disabled = True
    slider_red2_s.disabled = True
    cap.release()
    # 清除画面
    image_processed.value = b''
    image_mask.value = b''

### 组件设置

In [29]:
# 视频源
cap = cv2.VideoCapture('video/red-green.mp4')
if not cap.isOpened():
    raise IOError("无法打开视频文件")

# -----------------------image控件--------------------------------
# 掩膜图像， 和圈出区域后的图像
image_processed = widgets.Image(format='jpg', width=640, height=480)
image_mask = widgets.Image(format='jpg', width=640, height=480)
# ----------------------slider控件--------------------------------
# red1和red2的范围
slider_red1_h = widgets.IntRangeSlider(value=[0, 15], min=0, max=180, step=1, description='Red1 H:', continuous_update=False)
slider_red1_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, step=1, description='Red1 S:', continuous_update=False)
slider_red1_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, step=1, description='Red1 V:', continuous_update=False)
slider_red2_h = widgets.IntRangeSlider(value=[165, 180], min=0, max=180, step=1, description='Red2 H:', continuous_update=False)
slider_red2_s = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, step=1, description='Red2 S:', continuous_update=False)
slider_red2_v = widgets.IntRangeSlider(value=[80, 255], min=0, max=255, step=1, description='Red2 V:', continuous_update=False)
# lowest的范围
slider_lowest = widgets.IntSlider(value=30, min=0, max=100, step=1, description='LOWEST:', continuous_update=False)

# -------------------------------button控件--------------------------
# 控制关闭和打开
toggle_button = widgets.ToggleButton(value=False, description="Play/Pause", icon='play')
close_button = widgets.Button(description="关闭", icon='times', button_style='danger')
toggle_button.observe(toggle_play, names='value')
close_button.on_click(close_video)
# -----------------------------回调函数--------------------------------
def refresh_on_change(change=None):
    with frame_lock:
        current = cap.get(cv2.CAP_PROP_POS_FRAMES)
        cap.set(cv2.CAP_PROP_POS_FRAMES, max(0, current - 1))
        ret, frame = cap.read()
        if ret:
            temp = [
                [slider_red1_h.value[0], slider_red1_s.value[0],slider_red1_v.value[0]],
                [slider_red1_h.value[1], slider_red1_s.value[1],slider_red1_v.value[1]],
                [slider_red2_h.value[0], slider_red2_s.value[0],slider_red2_v.value[0]],
                [slider_red2_h.value[1], slider_red2_s.value[1],slider_red2_v.value[1]],
            ]
            processed, mask = process_frame(frame, temp, lowest=slider_lowest.value)
            image_processed.value = to_jpeg_bytes(processed)
            image_mask.value = to_jpeg_bytes(mask, is_mask=True)
# ----------------------------绑定回调函数和控件------------------------
for s in [slider_lowest, slider_red1_h, slider_red2_h, slider_red1_s, slider_red1_v,slider_red2_v]:
    s.observe(refresh_on_change, names='value')

### 布局

In [30]:
group_red1 = widgets.HBox([slider_red1_h, slider_red2_h])
group_red2 = widgets.HBox([slider_red1_s, slider_red2_s])
group_red3 = widgets.HBox([slider_red1_v, slider_red2_v])
button_bar = widgets.HBox([toggle_button, close_button])
controls = widgets.VBox([button_bar, slider_lowest, group_red1, group_red2, group_red3])
image = widgets.HBox([image_processed, image_mask])

### 启动

In [31]:
display(controls, image)
refresh_on_change()